In [4]:
# Install dependencies
!pip install -q langchain langchain-community faiss-cpu sentence-transformers pypdf



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip install -q langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.8 MB/s eta 0:00:00


In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
import os

In [7]:
# Load the Constitution PDF
pdf_path = "/content/The_Constitution_of_Kenya_2010.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [8]:
#  Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(documents)

In [9]:
#  Create free embeddings (using MiniLM)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-3253230758.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
#  Store in FAISS vector database
vectorstore = FAISS.from_documents(docs, embeddings)

In [14]:
os.environ["GROQ_API_KEY"] = "my_api_key"

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    api_key=os.environ["GROQ_API_KEY"]
)

In [15]:
#  Build RAG pipeline
qa = RetrievalQA.from_chain_type(
    llm=llm,#uses Groq’s LLM to combine the chunks + user query into a final answer.
    retriever=vectorstore.as_retriever(search_kwargs={"k":3}),#pulls the top 3 relevant Constitution chunks.
    chain_type="stuff"
)

In [16]:
#  Test it
query = "What does the Constitution say about the right to life?"
answer = qa.run(query)
print("Q:", query)
print("A:", answer)

Q: What does the Constitution say about the right to life?
A: According to the Constitution of Kenya, 2010, the right to life is stated in Article 26. It says:

"(1) Every person has the right to life.
(2) The life of a person begins at conception.
(3) A person shall not be deprived of life intentionally, except to the extent authorised by this Constitution or other written law.
(4) Abortion is not permitted unless, in the opinion of a trained health professional, there is need for emergency treatment, or the life or health of the mother is in danger, or if permitted by any other written law."

This article establishes that every person has the right to life, and that life begins at conception. It also sets out the circumstances under which a person's life can be intentionally deprived, which are limited to situations authorised by the Constitution or other written law.


In [17]:
# Install Gradio if not already installed
!pip install gradio


In [18]:
import gradio as gr

# Define chatbot function
def chat_with_constitution(user_input, history=[]):
    try:
        answer = qa.run(user_input)   # qa is your RAG RetrievalQA chain
    except Exception as e:
        answer = f"Error: {e}"
    return answer

In [19]:
# Create a Gradio ChatInterface
iface = gr.ChatInterface(
    fn=chat_with_constitution,
    title=" Chat with the Constitution of Kenya",
    description="Ask questions about the Constitution of Kenya (2010).",
    theme="soft"
)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [20]:
# Launch app
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63931e07b6dd576ef5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
